<a href="https://colab.research.google.com/github/FGalvao77/Intro-Redes-Adversarias-Generativas-GAN---Generative-Adversarial-Network-/blob/main/Tutorial_GAN_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutorial GAN mnist**

---



In [ ]:
# baixando o repositório do GitHub
!git clone https://github.com/bstriner/keras_adversarial.git

fatal: destination path 'keras_adversarial' already exists and is not an empty directory.


Mudança feita no setup
- de: long_description = open('README.rst').read()
- para: long_description = open('keras_adversarial/README.rst).read()

In [ ]:
# !python keras_adversarial/setup.py

In [ ]:
# importando as bibliotecas
import keras_adversarial

In [ ]:
# instanciando a versão necessário do keras
!pip install keras==2.1.2

In [ ]:
# instanciando a versão necessário do tensordflow
!pip install tensorflow==1.13.1

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import matplotlib as mpl  

mpl.use('Agg')

In [ ]:
# importando as demais bibliotecas e funções
import pandas as pd
import numpy as np
import os

from keras.layers import Reshape, Flatten, LeakyReLU, Activation
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
import keras.backend as K

from keras_adversarial.keras_adversarial.image_grid_callback import ImageGridCallback
from keras_adversarial.keras_adversarial import (AdversarialModel, simple_gan, 
                                                 gan_targets, normal_latent_sampling, 
                                                 AdversarialOptimizerSimultaneous)
from keras_adversarial.keras_adversarial.legacy import l1l2, Dense, fit
from keras_adversarial.examples.mnist_utils import mnist_data

Using TensorFlow backend.


In [ ]:
# biblioteca para ignorar mensagens de alerta
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# função geradora
def model_generator(latent_dim, input_shape, hidden_dim=1024, reg=lambda: l1l2(1e-5, 1e-5)):
    return Sequential([
                       Dense(int(hidden_dim / 4), name='generator_h1', input_dim=latent_dim, W_regularizer=reg()),
                       LeakyReLU(0.2),
                       Dense(int(hidden_dim / 2), name='generator_h2', W_regularizer=reg()),
                       LeakyReLU(0.2),
                       Dense(hidden_dim, name='generator_h3', W_regularizer=reg()),
                       LeakyReLU(0.2),
                       Dense(np.prod(input_shape), name='generator_x_flat', W_regularizer=reg()),
                       Activation('sigmoid'),
                       Reshape(input_shape, name='generator_x')
    ], name='generator')


In [ ]:
# função discriminadora
def model_discriminator(input_shape, hidden_dim=1024, reg=lambda: l1l2(1e-5, 1e-5), output_activation='sigmoid'):
    return Sequential([
                       Flatten(name='discriminator_flatten', input_shape=input_shape),
                       Dense(hidden_dim, name='discriminator_h1', W_regularizer=reg()),
                       LeakyReLU(0.2),
                       Dense(int(hidden_dim / 2), name='discriminator_h2', W_regularizer=reg()),
                       LeakyReLU(0.2),
                       Dense(int(hidden_dim / 4), name='discriminator_h3', W_regularizer=reg()),
                       LeakyReLU(0.2),
                       Dense(1, name='discriminator_y', W_regularizer=reg()),
                       Activation(output_activation)
    ], name='discriminator')

In [ ]:
# função de exemplo de GAN
def example_gan(
    adversarial_optimizer, path, opt_g, opt_d, nb_epoch, generator, 
    discriminator, latent_dim, targets=gan_targets, loss='binary_crossentropy'
    ):
    
    csvpath=os.path.join(path, 'history.csv')

    if os.path.exists(csvpath):
        print('Already exists: {}'.format(csvpath))
        return

    print('Training: {}'.format(csvpath))
    # gan (x --> yfake, yreal), z generated on GPU
    gan = simple_gan(
        generator, discriminator, 
        normal_latent_sampling((latent_dim,)))

    # print summary of models
    generator.summary()
    discriminator.summary()
    gan.summary()

    # build adversarial model
    model = AdversarialModel(
        base_model=gan,
        player_params=[generator.trainable_weights, discriminator.trainable_weights],
        player_names=['generator', 'discriminator'] 
    )
    
    model.adversarial_compile(
        adversarial_optimizer=adversarial_optimizer,
        player_optimizers=[opt_g, opt_d],
        loss=loss
    )

    # create callbacks to generate images
    zsamples = np.random.normal(size=(10 * 10, latent_dim))

    def generator_sampler():
        return generator.predict(zsamples).reshape((10, 10, 28, 28))

    generator_cb = ImageGridCallback(
        os.path.join(path, 'epoch-{:03d}.png'), 
        generator_sampler
    )

    # train model
    xtrain, xtest = mnist_data()
    
    y = targets(xtrain.shape[0])
    ytest = targets(xtest.shape[0])

    callbacks = [generator_cb]

    if K.backend() == 'tensorflow':
        callbacks.append(
            TensorBoard(log_dir=os.path.join(path, 'logs'), 
                        histogram_freq=0, 
                        write_graph=True, 
                        write_images=True)
        )

    history = fit(model, x=xtrain, y=y, 
                  validation_data=(xtest, ytest), 
                  callbacks=callbacks,
                  nb_epoch=nb_epoch, 
                  batch_size=32)
    
    # save history to CSV
    df = pd.DataFrame(history.history)
    df.to_csv(csvpath)

    # save models
    generator.save(
        os.path.join(path, 'generator.h5')
    )
    model_discriminator(
        os.path.join(path, 'discriminator.h5')
    )

In [ ]:
# função principal para execução das demais funções
def main():
    # z \in R^100
    latent_dim = 100

    # x \in R^(28x28)
    input_shape = (28,28)

    # generator (z --> x)
    generator = model_generator(latent_dim, input_shape)

    # discriminator (x --> y)
    discriminator = model_discriminator(input_shape)

    example_gan(
        AdversarialOptimizerSimultaneous(), 'output/gan',
        opt_g=Adam(1e-4, decay=1e-4),
        opt_d=Adam(1e-3, decay=1e-4),
        nb_epoch=100, 
        generator=generator, 
        discriminator=discriminator,
        latent_dim=latent_dim)

In [ ]:
import __main__

# executando a função
if __name__ == '__main__':
    main()




Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Training: output/gan/history.csv

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generator_h1 (Dense)         (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
generator_h2 (Dense)         (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
generator_h3 (Dense)         (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)

ValueError: ignored

In [ ]:
# importando as imagens para um arquivo zipado
!zip -r /content/file.zip /content/output

  adding: content/output/ (stored 0%)
  adding: content/output/gan/ (stored 0%)
  adding: content/output/gan/epoch-097.png (deflated 10%)
  adding: content/output/gan/epoch-086.png (deflated 9%)
  adding: content/output/gan/epoch-074.png (deflated 9%)
  adding: content/output/gan/epoch-067.png (deflated 10%)
  adding: content/output/gan/epoch-050.png (deflated 9%)
  adding: content/output/gan/epoch-001.png (deflated 9%)
  adding: content/output/gan/epoch-072.png (deflated 9%)
  adding: content/output/gan/epoch-022.png (deflated 8%)
  adding: content/output/gan/epoch-077.png (deflated 10%)
  adding: content/output/gan/epoch-019.png (deflated 8%)
  adding: content/output/gan/epoch-087.png (deflated 9%)
  adding: content/output/gan/epoch-044.png (deflated 8%)
  adding: content/output/gan/epoch-036.png (deflated 9%)
  adding: content/output/gan/epoch-032.png (deflated 8%)
  adding: content/output/gan/logs/ (stored 0%)
  adding: content/output/gan/logs/events.out.tfevents.1642868779.de135db

In [20]:
# realizando o download do arquivo
from google.colab import files
files.download('/content/file.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>